In [ ]:
!pip install torch sentencepiece pyyaml nltk wandb tqdm -q

In [ ]:
import wandb
wandb.login(key='abc')

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: murtaza-0903 (murtaza-0903-heriot-watt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
!unzip -q nmt_colab.zip
!ls -la

total 1860
drwxr-xr-x 1 root root    4096 Nov 22 14:06 .
drwxr-xr-x 1 root root    4096 Nov 22 14:01 ..
drwxr-xr-x 4 root root    4096 Nov 20 14:30 .config
drwxr-xr-x 2 root root    4096 Nov 21 11:23 multi30k_data
-rw-r--r-- 1 root root 1874643 Nov 22 14:06 nmt_colab.zip
drwxr-xr-x 3 root root    4096 Nov 22 14:06 part1
-rw-r--r-- 1 root root    1876 Nov 21 12:23 requirements.txt
drwxr-xr-x 1 root root    4096 Nov 20 14:30 sample_data


In [ ]:
import torch
print(f"✓ GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NO GPU!'}")

✓ GPU: Tesla T4


In [ ]:
%cd part1

/content/part1


In [ ]:
import torch
print(f"\n{'='*50}")
print(f"✓ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"{'='*50}\n")


✓ GPU Available: True
✓ GPU Name: Tesla T4
✓ GPU Memory: 15.83 GB



In [ ]:
import os
print("Checking data files...")
data_files = ['../multi30k_data/train.en', '../multi30k_data/train.fr',
              '../multi30k_data/val.en', '../multi30k_data/val.fr']
for f in data_files:
    exists = "✓" if os.path.exists(f) else "✗"
    print(f"{exists} {f}")

Checking data files...
✓ ../multi30k_data/train.en
✓ ../multi30k_data/train.fr
✓ ../multi30k_data/val.en
✓ ../multi30k_data/val.fr


In [ ]:
model_size = "large"  # Change to "medium" or "large" for other experiments

configs = {
    "small": {"embed_size": 16, "hidden_size": 16},
    "medium": {"embed_size": 64, "hidden_size": 64},
    "large": {"embed_size": 128, "hidden_size": 128}
}

with open('model_config.yaml', 'w') as f:
    f.write(f"""embed_size: {configs[model_size]['embed_size']}
hidden_size: {configs[model_size]['hidden_size']}
dropout_rate: 0.2
""")

print(f"\n🚀 Training {model_size.upper()} model ({configs[model_size]['embed_size']})")



🚀 Training LARGE model (128)


In [ ]:
# Change this to run different experiments:
experiment_name = "lr-low"  # Options: "baseline", "lr-low", "dropout-high", "batch-large"

# All experiment configurations
experiments = {
    "baseline": {
        "model": {"embed_size": 128, "hidden_size": 128, "dropout_rate": 0.2},
        "train": {"lr": 0.001, "train_batch_size": 32, "dev_batch_size": 128}
    },
    "lr-low": {
        "model": {"embed_size": 128, "hidden_size": 128, "dropout_rate": 0.2},
        "train": {"lr": 0.0005, "train_batch_size": 32, "dev_batch_size": 128}
    },
    "dropout-high": {
        "model": {"embed_size": 128, "hidden_size": 128, "dropout_rate": 0.4},
        "train": {"lr": 0.001, "train_batch_size": 32, "dev_batch_size": 128}
    },
    "batch-large": {
        "model": {"embed_size": 128, "hidden_size": 128, "dropout_rate": 0.2},
        "train": {"lr": 0.001, "train_batch_size": 128, "dev_batch_size": 256}
    }
}

# Get current experiment config
config = experiments[experiment_name]

# Write model_config.yaml
with open('model_config.yaml', 'w') as f:
    f.write(f"""embed_size: {config['model']['embed_size']}
hidden_size: {config['model']['hidden_size']}
dropout_rate: {config['model']['dropout_rate']}
""")

# Write train_config.yaml
with open('train_config.yaml', 'w') as f:
    f.write(f"""train_batch_size: {config['train']['train_batch_size']}
dev_batch_size: {config['train']['dev_batch_size']}
seed: 0
max_epoch: 30
patience: 1
lr: {config['train']['lr']}
lr_decay: 0.5
uniform_init: 0.1
valid_niter: 2000
log_every: 10
max_num_trial: 5
""")

print(f"\n{'='*60}")
print(f"🧪 Running experiment: {experiment_name.upper()}")
print(f"{'='*60}")
print(f"Model: embed={config['model']['embed_size']}, hidden={config['model']['hidden_size']}, dropout={config['model']['dropout_rate']}")
print(f"Train: lr={config['train']['lr']}, batch={config['train']['train_batch_size']}")
print(f"{'='*60}\n")


🧪 Running experiment: LR-LOW
Model: embed=128, hidden=128, dropout=0.2
Train: lr=0.0005, batch=32



In [ ]:
!python train.py \
    --model-config model_config.yaml \
    --train-config train_config.yaml \
    --wandb-project "nmt-coursework" \
    --wandb-run-name "large-{experiment_name}"

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Train dataset size: 29000
Dev dataset size: 1014
Load vocab from vocab/vocab.json
uniformly initialize parameters [-0.1, +0.1]
use device: cuda
Starting Training!
wandb: Currently logged in as: murtaza-0903 (murtaza-0903-heriot-watt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ setting up run 4houkdnj (0.2s)
wandb: ⣽ setting up run 4houkdnj (0.2s)
wandb: ⣾ setting up run 4houkdnj (0.2s)
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /content/part1/wandb/run-20251122_175426-4houkdnj
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run large-lr-low
wandb: ⭐️ View project at https://wandb.ai/murtaza-0903-heriot-watt-university/nmt-coursework
wandb: 🚀 View run at https://wandb.ai/murtaza-0903-heriot-watt-university/nmt-coursework/runs/4houkdnj


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.5 MB/s eta 0:00:00


In [ ]:
!python test.py --checkpoint-path models/nmt.model --test-source-file ../multi30k_data/test_2016_flickr.en --test-target-file ../multi30k_data/test_2016_flickr.fr

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Load test source sentences from ../multi30k_data/test_2016_flickr.en
Load test target sentences from ../multi30k_data/test_2016_flickr.fr
Load vocab from vocab/vocab.json
Load model from models/nmt.model
Generating Predictions: 100% 1000/1000 [00:08<00:00, 123.22it/s]
Corpus BLEU score = 50.40
Saving predictions to outputs/results.json
